In [1]:
import hail as hl
hl.init()

Running on Apache Spark version 2.4.5
SparkUI available at http://hail-m.us-central1-f.c.cncd-cncd.internal:4043
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.61-3c86d3ba497a
LOGGING: writing to /hail-20210206-1449-0.2.61-3c86d3ba497a.log


In [2]:
recode = {f"{i}":f"chr{i}" for i in (list(range(1, 23)) + ['X', 'Y'])}
mt1 = hl.import_vcf('gs://pgr_wes_30k/GRCh38/chr19_30k_grch38.vcf.bgz', reference_genome='GRCh38', contig_recoding=recode, force_bgz=True)
ds_result = hl.filter_intervals(mt1, [hl.parse_locus_interval('chr19:54816468-54830778', reference_genome='GRCh38')])
print('1')
participant_ids = ['CNCD_QNF332_QNF332','CNCD_KZ960_KZ960']
ds_result.rows().show()
print(ds_result.count())
ds_result1 = ds_result.filter_cols(hl.literal(participant_ids).contains(ds_result.s))
print(ds_result1.count())
ds_result1.rows().show()

print('2')
ds_result1 = hl.split_multi_hts(ds_result1, permit_shuffle=True)
ds_result1.rows().show()

print('3')
print('Annotate Heterozygous Alleles')
ds_result1 = ds_result1.annotate_rows(hets = hl.agg.filter(ds_result1.GT.is_het(),hl.agg.collect(ds_result1.s)))
ds_result1.count()
print('Annotate N Hets')
ds_result4 = ds_result1.annotate_rows(n_het = hl.agg.count_where(ds_result1.GT.is_het()))
ds_result4.count()
print('Annotate Homozygous Alleles')
ds_result3 = ds_result4.annotate_rows(homs = hl.agg.filter(ds_result4.GT.is_hom_var(),hl.agg.collect(ds_result4.s)))
ds_result3.count()
print('Annotating N Homs')
ds_result5 = ds_result3.annotate_rows(n_hom = hl.agg.count_where(ds_result3.GT.is_hom_var()))

ds_result5.rows().show()

data3 = ds_result5.annotate_rows(
    info=ds_result5.info.annotate(hets=ds_result5.hets,
                             n_het=ds_result5.n_het,
                             homs=ds_result5.homs,
                             n_hom=ds_result5.n_hom)
                                )

hl.export_vcf(data3, "gs://test1haseeb/KIR3DL1_20K.vcf")

1


2021-02-06 14:50:41 Hail: INFO: Coerced sorted dataset
2021-02-06 14:50:41 Hail: INFO: reading 1 of 224 data partitions


+----------------+------------+------+----------+
| locus          | alleles    | rsid |     qual |
+----------------+------------+------+----------+
| locus<GRCh38>  | array<str> | str  |  float64 |
+----------------+------------+------+----------+
| chr19:54816469 | ["G","A"]  | NA   | 3.27e+02 |
| chr19:54816471 | ["G","A"]  | NA   | 1.68e+03 |
| chr19:54816471 | ["G","T"]  | NA   | 1.68e+03 |
| chr19:54816472 | ["C","T"]  | NA   | 1.13e+03 |
| chr19:54816474 | ["C","T"]  | NA   | 5.60e+02 |
| chr19:54816475 | ["A","G"]  | NA   | 6.99e+05 |
| chr19:54816478 | ["C","T"]  | NA   | 1.97e+02 |
| chr19:54816479 | ["G","A"]  | NA   | 4.90e+03 |
| chr19:54816484 | ["T","A"]  | NA   | 6.59e+01 |
| chr19:54816491 | ["C","A"]  | NA   | 9.28e+03 |
+----------------+------------+------+----------+

+---------------------------------+---------+----------+---------------+
| filters                         | info.AC | info.aaf | info.callRate |
+---------------------------------+---------+----------+---------------+
| set<str>                        |   int32 |  float64 |       float64 |
+---------------------------------+---------+----------+---------------+
| {"VQSRTrancheSNP99.60to99.80"}  |       2 | 4.14e-05 |      7.83e-01 |
| {}                              |       5 | 8.11e-05 |      1.00e+00 |
| {}                              |       0 | 0.00e+00 |      1.00e+00 |
| {}                              |       1 | 2.07e-05 |      7.82e-01 |
| {"VQSRTrancheSNP99.90to99.95"}  |      11 | 2.28e-04 |      7.82e-01 |
| {"VQSRTrancheSNP99.95to100.00"} |    7733 | 1.69e-01 |      7.42e-01 |
| {"VQSRTrancheSNP99.80to99.90"}  |       2 | 4.14e-05 |      7.84e-01 |
| {}                              |       2 | 4.14e-05 |      7.84e-01 |
| {"VQSRTrancheSNP99.60to99.80"}  |       2 | 4.13e-05 |      7.84e-01 |
| {}                              |       6 | 9.73e-05 |      1.00e+00 |
+---------------------------------+---------+----------+---------------+

+------------+-------------------+------------+------------+---------------+
| info.hetRA | info.hg19_uid     | info.homAA | info.homRR | info.maxDepth |
+------------+-------------------+------------+------------+---------------+
|      int32 | str               |      int32 |      int32 |         int32 |
+------------+-------------------+------------+------------+---------------+
|          0 | "19:55327924:G:A" |          1 |      24133 |           506 |
|          1 | "19:55327926:G:A" |          2 |      30830 |           506 |
|          0 | "19:55327926:G:T" |          0 |      30831 |           506 |
|          1 | "19:55327927:C:T" |          0 |      24121 |           506 |
|          3 | "19:55327929:C:T" |          4 |      24096 |           506 |
|        223 | "19:55327930:A:G" |       3755 |      18894 |           506 |
|          2 | "19:55327933:C:T" |          0 |      24161 |           506 |
|          0 | "19:55327934:G:A" |          1 |      24180 |           506 |
|          0 | "19:55327939:T:A" |          1 |      24185 |           506 |
|          2 | "19:55327946:C:A" |          2 |      30825 |           506 |
+------------+-------------------+------------+------------+---------------+

+---------------+---------------+---------------+---------------+
| info.maxHetAB | info.maxHomAB | info.medDepth | info.medHetAB |
+---------------+---------------+---------------+---------------+
|       float64 |       float64 |         int32 |       float64 |
+---------------+---------------+---------------+---------------+
|            NA |            NA |            36 |            NA |
|            NA |            NA |            22 |            NA |
|            NA |            NA |            22 |            NA |
|            NA |            NA |            36 |            NA |
|      6.00e-01 |      1.00e+00 |            36 |      5.00e-01 |
|      8.22e-02 |      8.00e-01 |            38 |      8.22e-02 |
|            NA |            NA |            36 |      6.00e-01 |
|    

2021-02-06 14:51:04 Hail: INFO: Coerced sorted dataset
2021-02-06 14:51:04 Hail: INFO: reading 1 of 224 data partitions


(336, 30833)


...

2021-02-06 14:54:08 Hail: INFO: Coerced sorted dataset
2021-02-06 14:54:08 Hail: INFO: reading 1 of 224 data partitions
2021-02-06 14:54:17 Hail: INFO: Coerced sorted dataset
2021-02-06 14:54:17 Hail: INFO: reading 1 of 224 data partitions


Annotating N Homs


2021-02-06 14:54:32 Hail: INFO: Coerced sorted dataset
2021-02-06 14:54:32 Hail: INFO: reading 1 of 224 data partitions
2021-02-06 14:54:52 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-02-06 14:55:09 Hail: INFO: Coerced sorted dataset
2021-02-06 14:55:09 Hail: INFO: reading 1 of 224 data partitions


+----------------+------------+------+----------+
| locus          | alleles    | rsid |     qual |
+----------------+------------+------+----------+
| locus<GRCh38>  | array<str> | str  |  float64 |
+----------------+------------+------+----------+
| chr19:54816469 | ["G","A"]  | NA   | 3.27e+02 |
| chr19:54816471 | ["G","A"]  | NA   | 1.68e+03 |
| chr19:54816471 | ["G","T"]  | NA   | 1.68e+03 |
| chr19:54816472 | ["C","T"]  | NA   | 1.13e+03 |
| chr19:54816474 | ["C","T"]  | NA   | 5.60e+02 |
| chr19:54816475 | ["A","G"]  | NA   | 6.99e+05 |
| chr19:54816478 | ["C","T"]  | NA   | 1.97e+02 |
| chr19:54816479 | ["G","A"]  | NA   | 4.90e+03 |
| chr19:54816484 | ["T","A"]  | NA   | 6.59e+01 |
| chr19:54816491 | ["C","A"]  | NA   | 9.28e+03 |
+----------------+------------+------+----------+

+---------------------------------+---------+----------+---------------+
| filters                         | info.AC | info.aaf | info.callRate |
+---------------------------------+---------+----------+---------------+
| set<str>                        |   int32 |  float64 |       float64 |
+---------------------------------+---------+----------+---------------+
| {"VQSRTrancheSNP99.60to99.80"}  |       2 | 4.14e-05 |      7.83e-01 |
| {}                              |       5 | 8.11e-05 |      1.00e+00 |
| {}                              |       0 | 0.00e+00 |      1.00e+00 |
| {}                              |       1 | 2.07e-05 |      7.82e-01 |
| {"VQSRTrancheSNP99.90to99.95"}  |      11 | 2.28e-04 |      7.82e-01 |
| {"VQSRTrancheSNP99.95to100.00"} |    7733 | 1.69e-01 |      7.42e-01 |
| {"VQSRTrancheSNP99.80to99.90"}  |       2 | 4.14e-05 |      7.84e-01 |
| {}                              |       2 | 4.14e-05 |      7.84e-01 |
| {"VQSRTrancheSNP99.60to99.80"}  |       2 | 4.13e-05 |      7.84e-01 |
| {}                              |       6 | 9.73e-05 |      1.00e+00 |
+---------------------------------+---------+----------+---------------+

+------------+-------------------+------------+------------+---------------+
| info.hetRA | info.hg19_uid     | info.homAA | info.homRR | info.maxDepth |
+------------+-------------------+------------+------------+---------------+
|      int32 | str               |      int32 |      int32 |         int32 |
+------------+-------------------+------------+------------+---------------+
|          0 | "19:55327924:G:A" |          1 |      24133 |           506 |
|          1 | "19:55327926:G:A" |          2 |      30830 |           506 |
|          0 | "19:55327926:G:T" |          0 |      30831 |           506 |
|          1 | "19:55327927:C:T" |          0 |      24121 |           506 |
|          3 | "19:55327929:C:T" |          4 |      24096 |           506 |
|        223 | "19:55327930:A:G" |       3755 |      18894 |           506 |
|          2 | "19:55327933:C:T" |          0 |      24161 |           506 |
|          0 | "19:55327934:G:A" |          1 |      24180 |           506 |
|          0 | "19:55327939:T:A" |          1 |      24185 |           506 |
|          2 | "19:55327946:C:A" |          2 |      30825 |           506 |
+------------+-------------------+------------+------------+---------------+

+---------------+---------------+---------------+---------------+
| info.maxHetAB | info.maxHomAB | info.medDepth | info.medHetAB |
+---------------+---------------+---------------+---------------+
|       float64 |       float64 |         int32 |       float64 |
+---------------+---------------+---------------+---------------+
|            NA |            NA |            36 |            NA |
|            NA |            NA |            22 |            NA |
|            NA |            NA |            22 |            NA |
|            NA |            NA |            36 |            NA |
|      6.00e-01 |      1.00e+00 |            36 |      5.00e-01 |
|      8.22e-02 |      8.00e-01 |            38 |      8.22e-02 |
|            NA |            NA |            36 |      6.00e-01 |
|    

2021-02-06 14:55:45 Hail: WARN: export_vcf: ignored the following fields:
    'a_index' (row)
    'was_split' (row)
    'hets' (row)
    'n_het' (row)
    'homs' (row)
    'n_hom' (row)
2021-02-06 14:55:55 Hail: INFO: Coerced sorted dataset
2021-02-06 14:55:55 Hail: INFO: reading 1 of 224 data partitions
2021-02-06 14:56:30 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-02-06 14:56:39 Hail: INFO: Coerced sorted dataset
2021-02-06 14:56:39 Hail: INFO: reading 1 of 224 data partitions
2021-02-06 14:57:49 Hail: INFO: merging 1 files totalling 305.6K...
2021-02-06 14:57:50 Hail: INFO: while writing:
    gs://test1haseeb/KIR3DL1_20K.vcf
  merge time: 324.362ms
